# BTC Momentum Analysis — Module 02 Only

**Objetivo:** Produzir sinais de momentum a partir das bandas quantílicas do Módulo 02.

**Escopo:** Apenas arquivos `preds_T=*.parquet` com T ∈ {42,48,54,60}.

**Saídas:**
- Série temporal de scores de momentum (direção/volatilidade/confiança)
- Snapshots por horizonte T
- Gráficos de análise
- Relatório HTML
- Pré-checagens para Módulo 03

---

**Data de execução:** October 2, 2025  
**Timezone:** UTC

## 1. Setup & Configuration

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import yaml
from datetime import datetime, timedelta
import warnings
from scipy import stats
from typing import Dict, List, Tuple

warnings.filterwarnings('ignore')

# Configuração de plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Seed para reprodutibilidade
np.random.seed(42)

print("✅ Imports carregados")
print(f"📅 Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}")

✅ Imports carregados
📅 Data: 2025-10-02 15:58:53 UTC


### 1.1 Configuration Parameters

In [2]:
# Configuração do notebook
CONFIG = {
    'lookback_days': 180,           # janela para análises/percentis
    'horizons': [42, 48, 54, 60],   # horizontes de previsão (horas)
    'target_T_default': 48,         # horizonte padrão para gráficos
    'tilt_strength_hi': 1.2,        # |tilt_ratio| > este valor => direção forte
    'vol_hi_pct': 0.80,             # percentil de largura > 0.8 => vol alta
    'vol_lo_pct': 0.20,             # percentil de largura < 0.2 => vol baixa
    'recency_max_days': 7,          # máximo de dias desde última previsão
    
    'score_weights': {
        'directional': {'tilt': 0.6, 'slope': 0.4},
        'volatility': {'width_pct': 0.7, 'rv_delta': 0.3},
        'confidence': {'consistency': 0.5, 'stability': 0.5}
    },
    
    'export_paths': {
        'ts_table': 'data/processed/momentum/momentum_timeseries.parquet',
        'snapshot': 'data/processed/momentum/momentum_snapshot.csv',
        'report_html': 'data/processed/momentum/momentum_report.html',
        'charts_dir': 'data/processed/momentum/charts/'
    }
}

# Exibir configuração
print("⚙️  Configuração do Notebook:")
print("=" * 60)
for key, value in CONFIG.items():
    if key != 'score_weights' and key != 'export_paths':
        print(f"{key:20s}: {value}")

print("\n📊 Pesos dos Scores:")
for score_type, weights in CONFIG['score_weights'].items():
    print(f"  {score_type:15s}: {weights}")

print("\n✅ Configuração carregada")

⚙️  Configuração do Notebook:
lookback_days       : 180
horizons            : [42, 48, 54, 60]
target_T_default    : 48
tilt_strength_hi    : 1.2
vol_hi_pct          : 0.8
vol_lo_pct          : 0.2
recency_max_days    : 7

📊 Pesos dos Scores:
  directional    : {'tilt': 0.6, 'slope': 0.4}
  volatility     : {'width_pct': 0.7, 'rv_delta': 0.3}
  confidence     : {'consistency': 0.5, 'stability': 0.5}

✅ Configuração carregada


## 2. Data Discovery & Loading

### 2.1 Descoberta de Arquivos

In [3]:
# Descobrir arquivos de predições
data_dir = Path('data/processed/preds')
pred_files = sorted(data_dir.glob('preds_T=*.parquet'))

print(f"🔍 Buscando em: {data_dir}")
print(f"📁 Arquivos encontrados: {len(pred_files)}")
print("=" * 60)

if not pred_files:
    # Tentar diretório alternativo (notebooks)
    alt_dir = Path('notebooks/data/processed/preds')
    pred_files = sorted(alt_dir.glob('preds_T=*.parquet'))
    print(f"🔍 Tentando diretório alternativo: {alt_dir}")
    print(f"📁 Arquivos encontrados: {len(pred_files)}")

if pred_files:
    for f in pred_files:
        size_mb = f.stat().st_size / (1024*1024)
        print(f"  ✓ {f.name:30s} ({size_mb:6.2f} MB)")
else:
    print("❌ ERRO: Nenhum arquivo preds_T=*.parquet encontrado!")
    print("\nVerificando estrutura de diretórios...")
    if data_dir.exists():
        all_files = list(data_dir.glob('*'))
        print(f"\nArquivos em {data_dir}:")
        for f in all_files[:10]:  # Primeiros 10
            print(f"  - {f.name}")

# Buscar metadados opcionais
meta_file = data_dir / 'meta_pred.json'
qc_file = data_dir / 'qc_oos.json'

print(f"\n📋 Metadados:")
print(f"  meta_pred.json: {'✓ Encontrado' if meta_file.exists() else '✗ Não encontrado'}")
print(f"  qc_oos.json:    {'✓ Encontrado' if qc_file.exists() else '✗ Não encontrado'}")

print("\n✅ Descoberta concluída")

🔍 Buscando em: data/processed/preds
📁 Arquivos encontrados: 1
  ✓ preds_T=42.parquet             (  0.01 MB)

📋 Metadados:
  meta_pred.json: ✓ Encontrado
  qc_oos.json:    ✗ Não encontrado

✅ Descoberta concluída


### 2.2 Carregar e Concatenar Dados

In [4]:
# Carregar todos os arquivos de predição
if not pred_files:
    raise FileNotFoundError("❌ Nenhum arquivo de predição encontrado. Verifique o diretório de dados.")

dfs = []
for f in pred_files:
    try:
        df_temp = pd.read_parquet(f)
        dfs.append(df_temp)
        print(f"✓ Carregado {f.name}: {len(df_temp):,} linhas")
    except Exception as e:
        print(f"✗ Erro ao carregar {f.name}: {e}")

# Concatenar
if dfs:
    df_raw = pd.concat(dfs, ignore_index=True)
    print(f"\n📊 Dataset concatenado: {len(df_raw):,} linhas, {len(df_raw.columns)} colunas")
else:
    raise ValueError("❌ Nenhum arquivo foi carregado com sucesso")

# Exibir colunas disponíveis
print(f"\n📋 Colunas disponíveis:")
print(f"  {', '.join(sorted(df_raw.columns))}")

# Amostra dos dados
print(f"\n🔍 Primeiras linhas:")
display(df_raw.head())

print("\n✅ Dados carregados")

✓ Carregado preds_T=42.parquet: 1 linhas

📊 Dataset concatenado: 1 linhas, 12 colunas

📋 Colunas disponíveis:
  S0, T, h_days, p_25, p_50, p_75, p_med, q25, q50, q75, rvhat_ann, ts0

🔍 Primeiras linhas:


,ts0,T,h_days,S0,rvhat_ann,q25,p_25,q50,p_50,q75,p_75,p_med
0,2025-10-02 00:00:00+00:00,42,7.0,118822.1,NaN,-0.016067,116928.199661,0.010727,120103.593414,0.039374,123593.87164,120103.593414



✅ Dados carregados


## 3. Data Validation & Preprocessing

### 3.1 Validações Básicas

In [ ]:
print("🔍 Executando validações...\n")

# 1. Converter ts0 para UTC
if 'ts0' in df_raw.columns:
    df_raw['ts0'] = pd.to_datetime(df_raw['ts0'], utc=True)
    print(f"✓ Coluna 'ts0' convertida para UTC")
else:
    raise ValueError("❌ Coluna 'ts0' não encontrada")

# 2. Filtrar horizontes válidos
if 'T' in df_raw.columns:
    horizons_found = df_raw['T'].unique()
    print(f"\n📊 Horizontes encontrados: {sorted(horizons_found)}")
    
    invalid_T = [t for t in horizons_found if t not in CONFIG['horizons']]
    if invalid_T:
        print(f"⚠️  Horizontes inválidos (serão ignorados): {invalid_T}")
        df_raw = df_raw[df_raw['T'].isin(CONFIG['horizons'])].copy()
    
    print(f"✓ Dataset filtrado: {len(df_raw):,} linhas com T válido")
else:
    raise ValueError("❌ Coluna 'T' não encontrada")

# 3. Verificar campos obrigatórios
# Nota: Os arquivos têm q25, q50, q75 (não q05/q95)
required_cols = ['ts0', 'T', 'S0', 'q25', 'q50', 'q75']
optional_cols = ['p_25', 'p_50', 'p_75', 'p_med', 'rvhat_ann', 'h_days']

missing_cols = [col for col in required_cols if col not in df_raw.columns]

if missing_cols:
    raise ValueError(f"❌ Colunas obrigatórias ausentes: {missing_cols}")
else:
    print(f"\n✓ Todas as colunas obrigatórias presentes: {required_cols}")

# Verificar colunas opcionais disponíveis
available_optional = [col for col in optional_cols if col in df_raw.columns]
print(f"✓ Colunas opcionais disponíveis: {available_optional}")

# 4. Verificar monotonicidade dos quantis (q25 <= q50 <= q75)
monotone_check = (
    (df_raw['q25'] <= df_raw['q50']) &
    (df_raw['q50'] <= df_raw['q75'])
)

n_violations = (~monotone_check).sum()
pct_violations = 100 * n_violations / len(df_raw)

print(f"\n🔍 Monotonicidade dos quantis (q25 ≤ q50 ≤ q75):")
print(f"  Total de linhas: {len(df_raw):,}")
print(f"  Violações: {n_violations:,} ({pct_violations:.2f}%)")

if pct_violations > 1.0:
    print(f"  ⚠️  ATENÇÃO: {pct_violations:.2f}% de violações (limite: 1%)")
else:
    print(f"  ✓ Monotonicidade OK ({pct_violations:.4f}% violações)")

# 5. Verificar recência
latest_ts = df_raw['ts0'].max()
now = pd.Timestamp.now(tz='UTC')
days_since_last = (now - latest_ts).days

print(f"\n📅 Recência dos dados:")
print(f"  Última previsão: {latest_ts}")
print(f"  Dias desde última: {days_since_last}")
print(f"  Limite configurado: {CONFIG['recency_max_days']} dias")

if days_since_last > CONFIG['recency_max_days']:
    print(f"  ⚠️  ATENÇÃO: Dados podem estar desatualizados")
else:
    print(f"  ✓ Dados recentes")

# 6. Verificar NaNs em colunas críticas
print(f"\n🔍 NaNs em colunas críticas:")
for col in required_cols:
    n_nans = df_raw[col].isna().sum()
    if n_nans > 0:
        print(f"  ⚠️  {col}: {n_nans:,} NaNs ({100*n_nans/len(df_raw):.2f}%)")
    else:
        print(f"  ✓ {col}: sem NaNs")

print("\n✅ Validações concluídas")
print(f"\n📝 Nota: Dataset contém quantis q25, q50, q75 (não q05/q95)")

🔍 Executando validações...

✓ Coluna 'ts0' convertida para UTC

📊 Horizontes encontrados: [np.int64(42)]
✓ Dataset filtrado: 1 linhas com T válido


ValueError: ❌ Colunas obrigatórias ausentes: ['q05', 'q95']

### 3.2 Informações do Dataset

In [ ]:
# Resumo estatístico
print("📊 Resumo do Dataset")
print("=" * 60)
print(f"Total de linhas: {len(df_raw):,}")
print(f"Total de colunas: {len(df_raw.columns)}")
print(f"\nPeríodo dos dados:")
print(f"  Início: {df_raw['ts0'].min()}")
print(f"  Fim:    {df_raw['ts0'].max()}")
print(f"  Dias:   {(df_raw['ts0'].max() - df_raw['ts0'].min()).days}")

print(f"\n📈 Distribuição por horizonte T:")
for T in sorted(df_raw['T'].unique()):
    n = (df_raw['T'] == T).sum()
    pct = 100 * n / len(df_raw)
    print(f"  T={T}h: {n:6,} linhas ({pct:5.2f}%)")

print(f"\n💰 Estatísticas de S0 (preço spot):")
print(f"  Min:    ${df_raw['S0'].min():,.2f}")
print(f"  Média:  ${df_raw['S0'].mean():,.2f}")
print(f"  Mediana:${df_raw['S0'].median():,.2f}")
print(f"  Max:    ${df_raw['S0'].max():,.2f}")

# Estatísticas dos quantis disponíveis
print(f"\n📊 Estatísticas dos Quantis (valores relativos):")
quantile_cols = ['q25', 'q50', 'q75']
print(df_raw[quantile_cols].describe())

# Se tiver preços absolutos, mostrar também
if 'p_25' in df_raw.columns:
    print(f"\n💵 Estatísticas de Preços Absolutos (USD):")
    price_cols = ['p_25', 'p_50', 'p_75']
    available_price_cols = [c for c in price_cols if c in df_raw.columns]
    if available_price_cols:
        print(df_raw[available_price_cols].describe())

# Volatilidade realizada se disponível
if 'rvhat_ann' in df_raw.columns:
    print(f"\n📉 Estatísticas de Volatilidade Realizada Anualizada:")
    print(f"  Min:    {df_raw['rvhat_ann'].min():.4f}")
    print(f"  Média:  {df_raw['rvhat_ann'].mean():.4f}")
    print(f"  Mediana:{df_raw['rvhat_ann'].median():.4f}")
    print(f"  Max:    {df_raw['rvhat_ann'].max():.4f}")

print("\n✅ Informações do dataset exibidas")

## 4. Feature Engineering

### 4.1 Criar Bandas Absolutas e Quantis Adicionais

Como os dados contêm apenas q25, q50, q75, vamos:
1. Criar bandas de preço absolutas (p = S0 × q) se necessário
2. Estimar q05 e q95 usando a assimetria dos quantis existentes

In [ ]:
# Criar cópia para trabalhar
df = df_raw.copy()

print("🔧 Enriquecendo dataset com features derivadas...\n")

# 1. Garantir que temos preços absolutos p_25, p_50, p_75
if 'p_25' not in df.columns:
    df['p_25'] = df['S0'] * df['q25']
    print("✓ Criado p_25 = S0 × q25")
    
if 'p_50' not in df.columns:
    df['p_50'] = df['S0'] * df['q50']
    print("✓ Criado p_50 = S0 × q50")
    
if 'p_75' not in df.columns:
    df['p_75'] = df['S0'] * df['q75']
    print("✓ Criado p_75 = S0 × q75")

# 2. Estimar q05 e q95 usando extrapolação simétrica
# Assuma distribuição simétrica em torno de q50 (pode ser ajustado)
# q05 ≈ q50 - k*(q50-q25), onde k é fator de extrapolação
# q95 ≈ q50 + k*(q75-q50)

# Usar razão baseada na distância normal padrão
# Distância q25→q50: 0.25 da distribuição (z ≈ -0.674)
# Distância q05→q50: 0.45 da distribuição (z ≈ -1.645)
# Ratio: 1.645/0.674 ≈ 2.44

k_lower = 2.44  # extrapolação para q05
k_upper = 2.44  # extrapolação para q95

df['q05'] = df['q50'] - k_lower * (df['q50'] - df['q25'])
df['q95'] = df['q50'] + k_upper * (df['q75'] - df['q50'])

# Criar preços absolutos correspondentes
df['p_05'] = df['S0'] * df['q05']
df['p_95'] = df['S0'] * df['q95']

print(f"✓ Estimado q05 e q95 (extrapolação k={k_lower:.2f})")
print(f"✓ Criado p_05 e p_95")

# 3. Verificar sanidade dos quantis estimados
print(f"\n🔍 Validação dos quantis estimados:")
monotone_full = (
    (df['q05'] <= df['q25']) &
    (df['q25'] <= df['q50']) &
    (df['q50'] <= df['q75']) &
    (df['q75'] <= df['q95'])
)
n_ok = monotone_full.sum()
pct_ok = 100 * n_ok / len(df)
print(f"  Monotonicidade q05≤q25≤q50≤q75≤q95: {pct_ok:.2f}% OK")

if pct_ok < 95:
    print(f"  ⚠️  Ajustando quantis que violam monotonicidade...")
    # Forçar monotonicidade
    df.loc[df['q05'] > df['q25'], 'q05'] = df.loc[df['q05'] > df['q25'], 'q25'] * 0.99
    df.loc[df['q95'] < df['q75'], 'q95'] = df.loc[df['q95'] < df['q75'], 'q75'] * 1.01
    df['p_05'] = df['S0'] * df['q05']
    df['p_95'] = df['S0'] * df['q95']
    print(f"  ✓ Quantis ajustados para garantir monotonicidade")

print(f"\n✅ Dataset enriquecido: {len(df.columns)} colunas")
print(f"📋 Quantis disponíveis: q05, q25, q50, q75, q95")
print(f"💰 Preços disponíveis: p_05, p_25, p_50, p_75, p_95")